# Strands Agents with AgentCore Memory (Short-Term Memory)


## Introduction

This tutorial demonstrates how to build a **personal agent** using Strands agents with AgentCore **short-term memory** (Raw events). The agent remembers recent conversations in the session using `get_last_k_turns` and can continue conversations seamlessly when user returns.


### Tutorial Details

| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Short Term Conversational                                                        |
| Agent type          | Personal Agent                                                                   |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Sonnet 3.7                                                      |
| Tutorial components | AgentCore Short-term Memory, AgentInitializedEvent and MessageAddedEvent hooks   |
| Example complexity  | Beginner                                                                         |

You'll learn to:
- Use short-term memory for conversation continuity
- Retrieve last K conversation turns
- Web search tool for real-time information
- Initialize agents with conversation history

## Architecture
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## Prerequisites

- Python 3.10+
- AWS credentials with AgentCore Memory permissions
- AgentCore Memory role ARN
- Access to Amazon Bedrock models

Let's get started by setting up our environment!

## Step 1: Setup and Imports

In [1]:
!pip install -qr requirements.txt

In [2]:
import logging
from datetime import datetime

# Setup
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("personal-agent")

In [3]:
# Imports
import os
from strands import Agent, tool
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent
from bedrock_agentcore.memory import MemoryClient

# Configuration
REGION = os.getenv('AWS_REGION', 'us-west-2') # AWS region for the agent
ACTOR_ID = "user_123" # It can be any unique identifier (AgentID, User ID, etc.)
SESSION_ID = "personal_session_001" # Unique session identifier


## Step 2: Web Search Tool

First, let's create a simple web search tool for the agent.

In [5]:
from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

@tool
def websearch(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """Search the web for updated information.
    
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "Rate limit reached. Please try again later."
    except DDGSException as e:
        return f"Search error: {e}"
    except Exception as e:
        return f"Search error: {str(e)}"

logger.info("✅ Web search tool ready")

INFO:personal-agent:✅ Web search tool ready


## Step 3: Create Memory Resource
For short-term memory, we create a memory resource without any strategies. This stores raw conversation turns that can be retrieved with `get_last_k_turns`.


In [6]:
from botocore.exceptions import ClientError

# Initialize Memory Client
client = MemoryClient(region_name=REGION)
memory_name = "PersonalAgentMemory"

try:
    # Create memory resource without strategies (thus only access to short-term memory)
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=[],  # No strategies for short-term memory
        description="Short-term memory for personal agent",
        event_expiry_days=7, # Retention period for short-term memory. This can be upto 365 days.
    )
    memory_id = memory['id']
    logger.info(f"✅ Created memory: {memory_id}")
except ClientError as e:
    logger.info(f"❌ ERROR: {e}")
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Show any errors during memory creation
    logger.error(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.error(f"Failed to clean up memory: {cleanup_error}")

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:bedrock_agentcore.memory.client:Initialized MemoryClient for control plane: us-west-2, data plane: us-west-2
INFO:bedrock_agentcore.memory.client:Created memory: PersonalAgentMemory-4nR0vJGlSv
INFO:bedrock_agentcore.memory.client:Created memory PersonalAgentMemory-4nR0vJGlSv, waiting for ACTIVE status...
INFO:bedrock_agentcore.memory.client:Memory PersonalAgentMemory-4nR0vJGlSv is now ACTIVE (took 0 seconds)
INFO:personal-agent:✅ Created memory: PersonalAgentMemory-4nR0vJGlSv


## Step 4: Memory Hook

This step defines our custom `MemoryHookProvider` class that automates memory operations. Hooks are special functions that run at specific points in an agent's execution lifecycle. The memory hook we're creating serves two primary functions:
1. **To load recent conversation**: We use the `AgentInitializedEvent` hook will automatically load recent conversation history when the agent is initialized.
2. **To store the last message**: Stores new conversational message.

This creates a seamless memory experience without manual management.

In [16]:
class MemoryHookProvider(HookProvider):
    def __init__(self, memory_client: MemoryClient, memory_id: str):
        self.memory_client = memory_client
        self.memory_id = memory_id
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Load recent conversation history when agent starts"""
        try:
            # Load the last 5 conversation turns from memory
            # Get session info from agent state
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")
            
            if not actor_id or not session_id:
                logger.warning("Missing actor_id or session_id in agent state")
                return
            
            recent_turns = self.memory_client.get_last_k_turns(
                memory_id=self.memory_id,
                actor_id=actor_id,
                session_id=session_id,
                k=5
            )
            
            if recent_turns:
                # Format conversation history for context
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role']
                        content = message['content']['text']
                        context_messages.append(f"{role}: {content}")
                
                context = "\n".join(context_messages)
                # Add context to agent's system prompt.
                event.agent.system_prompt += f"\n\nRecent conversation:\n{context}"
                logger.info(f"✅ Loaded {len(recent_turns)} conversation turns")
                
        except Exception as e:
            logger.error(f"Memory load error: {e}")
    
    def on_message_added(self, event: MessageAddedEvent):
        """Store messages in memory"""
        messages = event.agent.messages
        try:
            # Get session info from agent state
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")

            if messages[-1]["content"][0].get("text"):
                self.memory_client.create_event(
                    memory_id=self.memory_id,
                    actor_id=actor_id,
                    session_id=session_id,
                    messages=[(messages[-1]["content"][0]["text"], messages[-1]["role"])]
                )
        except Exception as e:
            logger.error(f"Memory save error: {e}")
    
    def register_hooks(self, registry: HookRegistry):
        # Register memory hooks
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

## Step 5: Create Personal Agent with Web Search

In [17]:
def create_personal_agent():
    """Create personal agent with memory and web search"""
    agent = Agent(
        name="PersonalAssistantwithSTMemory",
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # or your preferred model
        system_prompt=f"""You are a helpful personal assistant with web search capabilities.
        
        You can help with:
        - General questions and information lookup
        - Web searches for current information
        - Personal task management
        
        When you need current information, use the websearch function.
        Today's date: {datetime.today().strftime('%Y-%m-%d')}
        Be friendly and professional.""",
        hooks=[MemoryHookProvider(client, memory_id)],
        tools=[websearch],
        state={"actor_id": ACTOR_ID, "session_id": SESSION_ID}
    )
    return agent

# Create agent
agent = create_personal_agent()
logger.info("✅ Personal agent created with memory and web search")

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:bedrock_agentcore.memory.client:Retrieved total of 7 events
INFO:personal-agent:✅ Loaded 4 conversation turns
INFO:personal-agent:✅ Personal agent created with memory and web search


#### Congratulations ! Your agent is ready ! :) 
## Lets test the Agent

In [18]:
# Test conversation with memory
print("=== First Conversation ===")
print(f"User: My name is Alex and I'm interested in learning about AI.")
print(f"Agent: ", end="")
agent("My name is Alex and I'm interested in learning about AI.")

INFO:bedrock_agentcore.memory.client:Created event: 0000001758434474000#14734c3d


=== First Conversation ===
User: My name is Alex and I'm interested in learning about AI.
Agent: Hi Alex! I'd be happy to help you learn about AI (Artificial Intelligence). AI is a fascinating field that's constantly evolving. Let me search for some up-to-date information for you.
Tool #1: websearch


INFO:bedrock_agentcore.memory.client:Created event: 0000001758434477000#f39fb148
INFO:primp:response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=what%20is%20artificial%20intelligence%20current%20developments%20in%20AI 200
INFO:primp:response: https://search.yahoo.com/search;_ylt=IZUI5cXewT2l4Hb_2MHLlFZd;_ylu=FsE0qDLfe2UUDyKwphr797_-JTJ6jSpBC0CixI2yLlVEWc0?p=what+is+artificial+intelligence+current+developments+in+AI 200


# Introduction to Artificial Intelligence (2025)

Based on the latest information, here's an overview of AI to help you get started:

## What is Artificial Intelligence?

Artificial Intelligence (AI) refers to computer systems designed to perform tasks that typically require human intelligence. These include:
- Problem-solving
- Learning from experience
- Understanding natural language
- Recognizing patterns and images
- Making decisions based on data

## Current AI Developments in 2025

According to IBM's latest trends report from May 2025, AI development continues to expand across multiple areas. Here are some key developments:

1. **Advanced AI Models**: AI has become increasingly sophisticated, now capable of performing complex tasks previously reserved for humans.

2. **Wide Application Range**: AI is being applied across diverse fields including:
   - Law
   - Customer support
   - Software development (coding)
   - Journalism
   - Healthcare
   - Climate science
   - Education



INFO:bedrock_agentcore.memory.client:Created event: 0000001758434487000#2b03b1d4


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "# Introduction to Artificial Intelligence (2025)\n\nBased on the latest information, here's an overview of AI to help you get started:\n\n## What is Artificial Intelligence?\n\nArtificial Intelligence (AI) refers to computer systems designed to perform tasks that typically require human intelligence. These include:\n- Problem-solving\n- Learning from experience\n- Understanding natural language\n- Recognizing patterns and images\n- Making decisions based on data\n\n## Current AI Developments in 2025\n\nAccording to IBM's latest trends report from May 2025, AI development continues to expand across multiple areas. Here are some key developments:\n\n1. **Advanced AI Models**: AI has become increasingly sophisticated, now capable of performing complex tasks previously reserved for humans.\n\n2. **Wide Application Range**: AI is being applied across diverse fields including:\n   - Law\n   - Customer sup

In [19]:
print(f"User: Can you search for the latest AI trends in 2025?")
print(f"Agent: ", end="")
agent("Can you search for the latest AI trends in 2025?")

INFO:bedrock_agentcore.memory.client:Created event: 0000001758434549000#19812815


User: Can you search for the latest AI trends in 2025?
Agent: I'll search for the latest AI trends in 2025 specifically to get you the most current information.
Tool #2: websearch


INFO:bedrock_agentcore.memory.client:Created event: 0000001758434551000#2b0cfe02
INFO:primp:response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=latest%20AI%20trends%202025%20artificial%20intelligence%20developments 200
INFO:primp:response: https://search.brave.com/search?q=latest+AI+trends+2025+artificial+intelligence+developments&source=web 200


# Latest AI Trends in 2025

Based on my search results, here are the most current AI trends shaping 2025:

## 1. Agentic AI
Multiple sources, including MIT Sloan Management Review and Gartner, identify agentic AI as the dominant trend of 2025. These are AI systems that can perform tasks independently with minimal human supervision. According to MIT Sloan, it's being called "2025's most trending AI trend," with significant excitement about AI agents that can accomplish tasks autonomously.

## 2. AI Reasoning Capabilities
Morgan Stanley reports that AI reasoning is one of the top frontier trends this year. Tech companies are focusing on refining cutting-edge AI applications that demonstrate more sophisticated reasoning abilities beyond simple pattern recognition.

## 3. Custom Silicon and Hardware
According to Morgan Stanley insights, the development of specialized AI hardware and custom silicon is a key trend driving innovation and performance improvements in 2025.

## 4. AI-Ready Data


INFO:bedrock_agentcore.memory.client:Created event: 0000001758434563000#12e5751b


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': '# Latest AI Trends in 2025\n\nBased on my search results, here are the most current AI trends shaping 2025:\n\n## 1. Agentic AI\nMultiple sources, including MIT Sloan Management Review and Gartner, identify agentic AI as the dominant trend of 2025. These are AI systems that can perform tasks independently with minimal human supervision. According to MIT Sloan, it\'s being called "2025\'s most trending AI trend," with significant excitement about AI agents that can accomplish tasks autonomously.\n\n## 2. AI Reasoning Capabilities\nMorgan Stanley reports that AI reasoning is one of the top frontier trends this year. Tech companies are focusing on refining cutting-edge AI applications that demonstrate more sophisticated reasoning abilities beyond simple pattern recognition.\n\n## 3. Custom Silicon and Hardware\nAccording to Morgan Stanley insights, the development of specialized AI hardware and custom 

In [20]:
print(f"User: I'm particularly interested in machine learning applications.")
print(f"Agent: ", end="")
agent("I'm particularly interested in machine learning applications.")

INFO:bedrock_agentcore.memory.client:Created event: 0000001758434579000#f6303f2b


User: I'm particularly interested in machine learning applications.
Agent: I'll search for the latest information on machine learning applications to provide you with the most current developments in this specific area of AI.
Tool #3: websearch


INFO:bedrock_agentcore.memory.client:Created event: 0000001758434582000#7cbcaf89
INFO:primp:response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=current%20machine%20learning%20applications%20trends%202025%20real-world%20uses 200
INFO:primp:response: https://www.mojeek.com/search?q=current+machine+learning+applications+trends+2025+real-world+uses 200


Let me search again for more current information about machine learning applications in 2025 specifically.
Tool #4: websearch


INFO:bedrock_agentcore.memory.client:Created event: 0000001758434587000#8201aa02
INFO:primp:response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=machine%20learning%20applications%202025%20industry%20applications%20ML%20trends%20healthcare%20finance%20manufacturing 200
INFO:primp:response: https://search.brave.com/search?q=machine+learning+applications+2025+industry+applications+ML+trends+healthcare+finance+manufacturing&source=web 200


# Machine Learning Applications in 2025: Current Trends and Real-World Uses

Based on the latest information, here are the most significant machine learning applications and trends in 2025:

## Healthcare Applications

According to recent reports from MobiDev (published just last week) and NIX United (from June 2025), healthcare is seeing transformative ML applications:

1. **Predictive Healthcare Analytics**: Google's DeepMind is analyzing electronic health records (EHRs) to forecast health risks and optimize treatment plans.

2. **Medical Imaging Analysis**: ML algorithms now detect anomalies in X-rays, MRIs, and CT scans with high accuracy, enabling earlier diagnosis of conditions like cancer.

3. **Market Growth**: The healthcare AI market is experiencing explosive growth, with a projected CAGR of 41.4% between 2020 and 2027, expected to reach $51.3 billion by 2027.

## Financial Services Applications

Helpware reports (also published last week) that financial services are leveragi

INFO:bedrock_agentcore.memory.client:Created event: 0000001758434601000#3076baaf


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': '# Machine Learning Applications in 2025: Current Trends and Real-World Uses\n\nBased on the latest information, here are the most significant machine learning applications and trends in 2025:\n\n## Healthcare Applications\n\nAccording to recent reports from MobiDev (published just last week) and NIX United (from June 2025), healthcare is seeing transformative ML applications:\n\n1. **Predictive Healthcare Analytics**: Google\'s DeepMind is analyzing electronic health records (EHRs) to forecast health risks and optimize treatment plans.\n\n2. **Medical Imaging Analysis**: ML algorithms now detect anomalies in X-rays, MRIs, and CT scans with high accuracy, enabling earlier diagnosis of conditions like cancer.\n\n3. **Market Growth**: The healthcare AI market is experiencing explosive growth, with a projected CAGR of 41.4% between 2020 and 2027, expected to reach $51.3 billion by 2027.\n\n## Financial 

## Test Memory Continuity

To test if our memory system is working correctly, we'll create a new instance of the agent and see if it can access the previously stored information:

In [21]:
# Create new agent instance (simulates user returning)
print("=== User Returns - New Session ===")
new_agent = create_personal_agent()

# Test memory continuity
print(f"User: What was my name again?")
print(f"Agent: ", end="")
new_agent("What was my name again?")

print(f"User: Can you search for more information about machine learning?")
print(f"Agent: ", end="")
new_agent("Can you search for more information about machine learning?")

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


=== User Returns - New Session ===


INFO:bedrock_agentcore.memory.client:Retrieved total of 17 events
INFO:personal-agent:✅ Loaded 5 conversation turns
INFO:bedrock_agentcore.memory.client:Created event: 0000001758434638000#e0d60c1e


User: What was my name again?
Agent: Based on our previous conversation, your name is Alex. You mentioned this when you introduced yourself saying "My name is Alex and I'm interested in learning about AI."

INFO:bedrock_agentcore.memory.client:Created event: 0000001758434640000#59925a91
INFO:bedrock_agentcore.memory.client:Created event: 0000001758434640000#c2c6a399


...

INFO:bedrock_agentcore.memory.client:Created event: 0000001758434643000#8ff6bb18
INFO:primp:response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=machine%20learning%20latest%20developments%20applications%20techniques 200
INFO:primp:response: https://www.mojeek.com/search?q=machine+learning+latest+developments+applications+techniques 200


Based on my search, here's some information about machine learning:

# Machine Learning: Latest Developments and Applications

## What is Machine Learning?
Machine learning is a subset of artificial intelligence that focuses on developing systems that can learn from and make decisions based on data, without being explicitly programmed for specific tasks.

## Recent Developments

1. **Advancements in Deep Learning Algorithms**: According to AISmasher, there have been significant breakthroughs in deep learning algorithms, enabling more sophisticated applications.

2. **Integration with Smart Cities**: MyAlphaFund reports that machine learning is playing a crucial role in the development of smart cities, helping urban environments become more connected and data-driven.

3. **Enhanced Computational Efficiency**: Research published in Frontiers in Energy Research highlights improvements in balancing computational efficiency and accuracy, particularly in applications like heat exchanger anal

INFO:bedrock_agentcore.memory.client:Created event: 0000001758434659000#9603ed95


 or focus on specific techniques like reinforcement learning or neural networks.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Based on my search, here's some information about machine learning:\n\n# Machine Learning: Latest Developments and Applications\n\n## What is Machine Learning?\nMachine learning is a subset of artificial intelligence that focuses on developing systems that can learn from and make decisions based on data, without being explicitly programmed for specific tasks.\n\n## Recent Developments\n\n1. **Advancements in Deep Learning Algorithms**: According to AISmasher, there have been significant breakthroughs in deep learning algorithms, enabling more sophisticated applications.\n\n2. **Integration with Smart Cities**: MyAlphaFund reports that machine learning is playing a crucial role in the development of smart cities, helping urban environments become more connected and data-driven.\n\n3. **Enhanced Computational Efficiency**: Research published in Frontiers in Energy Research highlights improvements in b

## View Stored Memory

In [23]:
# Check what's stored in memory
print("=== Memory Contents ===")
recent_turns = client.get_last_k_turns(
    memory_id=memory_id,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID,
    k=3 # Adjust k to see more or fewer turns
)

for i, turn in enumerate(recent_turns, 1):
    print(f"Turn {i}:")
    for message in turn:
        role = message['role']
        content = message['content']['text'][:100] + "..." if len(message['content']['text']) > 100 else message['content']['text']
        print(f"  {role}: {content}")
    print()

=== Memory Contents ===


INFO:bedrock_agentcore.memory.client:Retrieved total of 22 events


Turn 1:
  ASSISTANT: Based on my search, here's some information about machine learning:

# Machine Learning: Latest Deve...
  ASSISTANT: I'll search for more information about machine learning for you, Alex.

Turn 2:
  USER: Can you search for more information about machine learning?
  ASSISTANT: Based on our previous conversation, your name is Alex. You mentioned this when you introduced yourse...

Turn 3:
  USER: What was my name again?
  ASSISTANT: # Machine Learning Applications in 2025: Current Trends and Real-World Uses

Based on the latest inf...
  ASSISTANT: Let me search again for more current information about machine learning applications in 2025 specifi...
  ASSISTANT: I'll search for the latest information on machine learning applications to provide you with the most...



## Summary

This tutorial showed how to build a personal agent. You've learned:

- Creating memory resources without strategies
- Using `get_last_k_turns` for conversation history
- Adding web search capabilities to agents
- Implementing memory hooks for context loading

**Next Steps:**
- Add more sophisticated tools
- Implement long-term memory strategies
- Enhance search capabilities with multiple sources

## Cleanup (Optional)

In [ ]:
# Uncomment to delete memory resource
# client.delete_memory_and_wait(memory_id)
# logger.info(f"✅ Deleted memory: {memory_id}")